In [1]:
from ultralytics import YOLO
import numpy as np
import random
import torch
import os

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(150102)

In [ ]:
class Config:
    IMG_SIZE = 416
    BATCH_SIZE = 8
    EPOCHS = 10
    LEARNING_RATE = 3e-4
    NUM_CLASSES = 42
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    CLASSES = [
        "achichuk", "airan-katyk", "asip", "bauyrsak", "beshbarmak-w-kazy",
        "beshbarmak-wo-kazy", "chak-chak", "cheburek", "doner-lavash", "doner-nan",
        "hvorost", "irimshik", "kattama-nan", "kazy-karta", "kurt", "kuyrdak",
        "kymyz-kymyran", "lagman-fried", "lagman-w-soup", "lagman-wo-soup", "manty",
        "naryn", "nauryz-kozhe", "orama", "plov", "samsa", "shashlyk-chicken",
        "shashlyk-chicken-v", "shashlyk-kuskovoi", "shashlyk-kuskovoi-v",
        "shashlyk-minced-meat", "sheep-head", "shelpek", "shorpa", "soup-plain",
        "sushki", "suzbe", "taba-nan", "talkan-zhent", "tushpara-fried",
        "tushpara-w-soup", "tushpara-wo-soup"
    ]

    TRAIN_DATA = './CAFD_YOLO/train'
    VALIDATION_DATA = './CAFD_YOLO/val'
    TEST_DATA = './CAFD_YOLO/test'

cfg = Config()
yaml_path = './cafd.yaml'
yaml_content = f"""
train: {cfg.TRAIN_DATA}
val: {cfg.VALIDATION_DATA}
nc: {cfg.NUM_CLASSES}
names: {cfg.CLASSES}
"""

with open(yaml_path, 'w') as file:
    file.write(yaml_content)

In [ ]:
model = YOLO('yolov8m.pt')

model.train(
    data = yaml_path,
    epochs = cfg.EPOCHS,
    imgsz = cfg.IMG_SIZE,
    batch = cfg.BATCH_SIZE,
    lr0 = cfg.LEARNING_RATE,
    device = cfg.DEVICE,
    mosaic = True
)

model.export(format = 'torchscript')

New https://pypi.org/project/ultralytics/8.3.208 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.207 🚀 Python-3.11.13 torch-2.8.0+cu128 CPU (AMD Ryzen AI 7 350 w/ Radeon 860M)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./cafd.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=45, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=True, multi_scale=False, name=train9, nbs=64, nms=False, opset=None, o

KeyboardInterrupt: 

In [ ]:
# CUDA Diagnostic - Check why CUDA isn't working
import torch
import subprocess
import sys

print("🔍 CUDA Diagnostic Report")
print("=" * 50)

# 1. Check PyTorch CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"GPU {i} memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("❌ CUDA not available in PyTorch")

# 2. Check system CUDA installation
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print("\n✅ NVIDIA GPU detected:")
        print(result.stdout)
    else:
        print("\n❌ nvidia-smi not found or failed")
except Exception as e:
    print(f"\n❌ Error running nvidia-smi: {e}")

# 3. Check Python environment
print(f"\nPython version: {sys.version}")
print(f"Platform: {sys.platform}")

# 4. Check if running in WSL
try:
    with open('/proc/version', 'r') as f:
        version_info = f.read()
        if 'microsoft' in version_info.lower() or 'wsl' in version_info.lower():
            print("\n⚠️  Running in WSL - CUDA support may be limited")
        else:
            print("\n✅ Not running in WSL")
except:
    print("\n⚠️  Could not determine WSL status")

# 5. Test CUDA tensor creation
if torch.cuda.is_available():
    try:
        test_tensor = torch.tensor([1.0]).cuda()
        print(f"\n✅ CUDA tensor creation successful: {test_tensor}")
    except Exception as e:
        print(f"\n❌ CUDA tensor creation failed: {e}")
else:
    print("\n❌ Cannot test CUDA tensor creation - CUDA not available")

print("\n" + "=" * 50)
print("Diagnostic complete!")
